In [1]:
import psycopg2 
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg

In [2]:
conn = psycopg2.connect(
    host="fit3164db.cxkhqsoitzhb.ap-southeast-2.rds.amazonaws.com", 
    port=5432,
    user="postgres",
    password="fit3164d13edas",
    database = "testdb"
)
conn.autocommit = True
cursor = conn.cursor()

In [3]:
#actuals columns
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'actuals'")
acc_cols = cursor.fetchall()
acc_cols = [item[0] for item in acc_cols]
print(acc_cols)

['id', 'datetime', 'load', 'pressure', 'cloud_cover', 'humidity', 'temperature', 'wind_speed', 'wind_direction', 'date', 'month', 'hour', 'type_of_day', 'covid', 'holiday']


In [4]:
cursor.execute("select * from actuals")
actual_data = cursor.fetchall()

In [5]:
#forecasts columns
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'forecasts'")
fc_cols = cursor.fetchall()
fc_cols = [item[0] for item in fc_cols]
print(fc_cols)


['id', 'datetime', 'pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction', 'date', 'month', 'hour', 'type_of_day', 'covid', 'holiday']


In [6]:
cursor.execute("select * from forecasts")
forecast_data = cursor.fetchall()

In [7]:
cursor.close()
conn.close()

In [8]:
actuals = pd.DataFrame(actual_data, columns= acc_cols)
forecast = pd.DataFrame(forecast_data, columns=fc_cols)

In [9]:
# converting data types:
def convert_dtypes(df, actual=True):
    if actual:
        numeric_cols = ['month','hour', 'type_of_day','covid', 'holiday', 'load', 'pressure', 'cloud_cover', 'humidity', 'temperature', 'wind_speed', 'wind_direction']
    else:
        numeric_cols = ['month','hour', 'type_of_day','covid', 'holiday', 'pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction']
    
    date_cols = ['datetime', 'date']
    
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])
        
    for col in date_cols:
        df[col] = pd.to_datetime(df[col])
    
    return df
actuals = convert_dtypes(actuals)
forecast = convert_dtypes(forecast, False)

In [10]:
def calculate_mae(actual, predicted):
    return mean_absolute_error(actual, predicted)

def calculate_mape(actual, predicted):
    a = actual.to_numpy()
    p = predicted.to_numpy()

    return np.mean(np.abs((a - p) / a)) * 100

def calculate_rmse(actual, predicted):
    return math.sqrt(mean_squared_error(actual, predicted))

def get_error_measures(actual, predicted):
    result = "MAE: " + str(calculate_mae(actual, predicted)) + "\n"
    result += "MAPE: " + str(calculate_mape(actual, predicted)) + "\n"
    result += "RMSE: " + str(calculate_rmse(actual, predicted)) + "\n"
    return result

In [11]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [12]:
#lags for load 

def get_lag_leads(data, num_lags=168, forward_pred=48): 
    
    cols = ['pressure', 'cloud_cover', 'temperature', 'wind_speed',
       'wind_direction']

    #for load
    for i in range(1, num_lags + 1):    #gonna start from 2 bc need min 1 for skforecast
        data[f'load_lag_{i}'] = data['load'].shift(i) 

    #for lagged weather variables
    for c in cols:
        for i in range(1, num_lags+1):
            data[f'{c}_lag_{i}'] = data[c].shift(i)
        
    #for 48h forecasted weather ?
    # for c in cols: 
    #     for f in range(1, forward_pred + 1):    #lead 48 variables
    #         data[f'{c}_lead_{f}'] = data[c].shift(-f)

    data.dropna(inplace = True)    #drop nulls
    data = data.set_index('datetime')   #set index as datetime
    data = data.asfreq('H')

    return data
    

In [13]:
errors_df = pd.DataFrame(columns = ["start_date", "end_date", "mae", "mape", "rmse"])

In [14]:
data = pd.merge(actuals[['datetime','load']], forecast, on='datetime')
# data = data.rename(columns = {'datetime': 'ds','load': 'y'})

encode(data, 'hour', 24)
encode(data, 'month', 12)
encode(data, 'type_of_day', 31)

data = data.drop(['id', 'date', 'covid', 'holiday','month', 'hour', 'type_of_day'], axis=1)
data = data.rename(columns = {'type_of_day_sin': 'day_sin', 
                            'type_of_day_cos': 'day_cos'})

In [15]:
data.columns

Index(['datetime', 'load', 'pressure', 'cloud_cover', 'temperature',
       'wind_speed', 'wind_direction', 'hour_sin', 'hour_cos', 'month_sin',
       'month_cos', 'day_sin', 'day_cos'],
      dtype='object')

If we use lags=50, we won't be starting at 00:00:00, we will be starting at 02:00:00

In [16]:
data.head()

,datetime,load,pressure,cloud_cover,temperature,wind_speed,wind_direction,hour_sin,hour_cos,month_sin,month_cos,day_sin,day_cos
0,2017-03-18 00:00:00,1.031472e+06,1011.0,3.0,14.0,2.0,307.0,0.000000,1.000000,1.0,6.123234e-17,0.937752,0.347305
1,2017-03-18 01:00:00,1.007206e+06,1011.0,4.0,14.0,2.0,215.0,0.258819,0.965926,1.0,6.123234e-17,0.937752,0.347305
2,2017-03-18 02:00:00,9.861084e+05,1011.0,4.0,14.0,1.0,123.0,0.500000,0.866025,1.0,6.123234e-17,0.937752,0.347305
3,2017-03-18 03:00:00,9.707610e+05,1011.0,4.0,14.0,1.0,31.0,0.707107,0.707107,1.0,6.123234e-17,0.937752,0.347305
4,2017-03-18 04:00:00,9.622584e+05,1011.0,4.0,14.0,1.0,138.0,0.866025,0.500000,1.0,6.123234e-17,0.937752,0.347305


In [17]:
data = get_lag_leads(data)

C:\Users\danie\AppData\Local\Temp\ipykernel_11036\3072313938.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'load_lag_{i}'] = data['load'].shift(i)
C:\Users\danie\AppData\Local\Temp\ipykernel_11036\3072313938.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'load_lag_{i}'] = data['load'].shift(i)
C:\Users\danie\AppData\Local\Temp\ipykernel_11036\3072313938.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

In [18]:
data.columns[:50]

Index(['load', 'pressure', 'cloud_cover', 'temperature', 'wind_speed',
       'wind_direction', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
       'day_sin', 'day_cos', 'load_lag_1', 'load_lag_2', 'load_lag_3',
       'load_lag_4', 'load_lag_5', 'load_lag_6', 'load_lag_7', 'load_lag_8',
       'load_lag_9', 'load_lag_10', 'load_lag_11', 'load_lag_12',
       'load_lag_13', 'load_lag_14', 'load_lag_15', 'load_lag_16',
       'load_lag_17', 'load_lag_18', 'load_lag_19', 'load_lag_20',
       'load_lag_21', 'load_lag_22', 'load_lag_23', 'load_lag_24',
       'load_lag_25', 'load_lag_26', 'load_lag_27', 'load_lag_28',
       'load_lag_29', 'load_lag_30', 'load_lag_31', 'load_lag_32',
       'load_lag_33', 'load_lag_34', 'load_lag_35', 'load_lag_36',
       'load_lag_37', 'load_lag_38'],
      dtype='object')

In [19]:
import xgboost as xgb

In [72]:
exog = data.drop('load', axis=1)
target = data['load']

In [254]:
#idg the point of splitting into train and valid actually but sure lol
train_size = 30089
train_features, test_features = exog[:train_size], exog[train_size:train_size+12]
train_target, test_target = target[:train_size], target[train_size:train_size+12]

In [88]:
#recursive multi-step forecasting 
#==============================================================================
horizon = 48 #predict 48 hours

#this the model
model = xgb.XGBRegressor()  

def sliding_window(data, model=xgb.XGBRegressor(), horizon=48, slide_by=24, window_size=96):      
    
    first_timestep = 0
    last_timestep = window_size

    data_features = data.drop(['load'], axis=1)
    data_target = data['load']

    mae = []
    mape = []
    rmse = []

    windows = 0

    while last_timestep <= data.shape[0]:

        windows += 1
        
        #current window forecast features
        #over whole data? or over training only?
        window_train_exog = data_features.iloc[first_timestep: last_timestep]
        window_train_target = data_target.iloc[first_timestep: last_timestep]

        print(f'Fitting window {windows}...')
        model.fit(window_train_exog, window_train_target)

        window_mae = 0
        window_mape = 0
        window_rmse = 0
        
        
        #multi-step forecasting for the next 48 hours
        #create a table to store the forecasted values
        s = first_timestep + window_size
        l = s + horizon
        forecasted_values = pd.Series(data=np.zeros(horizon), index=data.iloc[s:l].index)

        for i in range(horizon):
            
            pred_start = first_timestep + window_size + i
            pred_end = pred_start + horizon + i

            #one time point
            forecast_features = data_features.iloc[pred_start: pred_start+1]

            #predict
            forecast = model.predict(forecast_features)

            #record the forecasted values
            forecasted_values[i] = forecast[0]

            #update the lag values with the predicted lag
            for lag in range(168,1,-1):
                forecast_features[f'load_lag_{lag}'] = forecast_features[f'load_lag_{lag-1}']
            forecast_features['load_lag_1'] = forecast 


        start_index = first_timestep + window_size
        end_index = start_index + horizon
        forecast_actuals = data_target.iloc[start_index:end_index]

        #error calculate
        window_mae += calculate_mae(forecast_actuals, forecasted_values)
        window_mape += calculate_mape(forecast_actuals, forecasted_values)
        window_rmse += calculate_rmse(forecast_actuals, forecasted_values)
            

        mae.append(window_mae/horizon)
        mape.append(window_mape/horizon)
        rmse.append(window_rmse/horizon)


        first_timestep += slide_by
        last_timestep += slide_by

        error_metrics = {}
        error_metrics['mae'] = mae
        error_metrics['mape'] = mape
        error_metrics['rmse'] = rmse

    return error_metrics
    

In [92]:
exog.head()

,pressure,cloud_cover,temperature,wind_speed,wind_direction,hour_sin,hour_cos,month_sin,month_cos,day_sin,...,wind_direction_lag_159,wind_direction_lag_160,wind_direction_lag_161,wind_direction_lag_162,wind_direction_lag_163,wind_direction_lag_164,wind_direction_lag_165,wind_direction_lag_166,wind_direction_lag_167,wind_direction_lag_168
datetime,,,,,,,,,,,,,,,,,,,,,
2017-03-25 00:00:00,1010.0,0.0,8.0,10.0,328.0,0.000000,1.000000,1.0,6.123234e-17,0.937752,...,256.0,288.0,320.0,352.0,245.0,138.0,31.0,123.0,215.0,307.0
2017-03-25 01:00:00,1010.0,0.0,8.0,10.0,219.0,0.258819,0.965926,1.0,6.123234e-17,0.937752,...,263.0,256.0,288.0,320.0,352.0,245.0,138.0,31.0,123.0,215.0
2017-03-25 02:00:00,1010.0,0.0,8.0,9.0,111.0,0.500000,0.866025,1.0,6.123234e-17,0.937752,...,271.0,263.0,256.0,288.0,320.0,352.0,245.0,138.0,31.0,123.0
2017-03-25 03:00:00,1010.0,0.0,8.0,9.0,2.0,0.707107,0.707107,1.0,6.123234e-17,0.937752,...,278.0,271.0,263.0,256.0,288.0,320.0,352.0,245.0,138.0,31.0
2017-03-25 04:00:00,1011.0,0.0,10.0,10.0,4.0,0.866025,0.500000,1.0,6.123234e-17,0.937752,...,280.0,278.0,271.0,263.0,256.0,288.0,320.0,352.0,245.0,138.0


In [96]:
exog.loc['2017-03-25 00:00:00', 'pressure'] = exog.loc['2017-03-25 01:00:00', 'wind_speed']

In [97]:
exog.head()

,pressure,cloud_cover,temperature,wind_speed,wind_direction,hour_sin,hour_cos,month_sin,month_cos,day_sin,...,wind_direction_lag_159,wind_direction_lag_160,wind_direction_lag_161,wind_direction_lag_162,wind_direction_lag_163,wind_direction_lag_164,wind_direction_lag_165,wind_direction_lag_166,wind_direction_lag_167,wind_direction_lag_168
datetime,,,,,,,,,,,,,,,,,,,,,
2017-03-25 00:00:00,10.0,0.0,8.0,10.0,328.0,0.000000,1.000000,1.0,6.123234e-17,0.937752,...,256.0,288.0,320.0,352.0,245.0,138.0,31.0,123.0,215.0,307.0
2017-03-25 01:00:00,1010.0,0.0,8.0,10.0,219.0,0.258819,0.965926,1.0,6.123234e-17,0.937752,...,263.0,256.0,288.0,320.0,352.0,245.0,138.0,31.0,123.0,215.0
2017-03-25 02:00:00,1010.0,0.0,8.0,9.0,111.0,0.500000,0.866025,1.0,6.123234e-17,0.937752,...,271.0,263.0,256.0,288.0,320.0,352.0,245.0,138.0,31.0,123.0
2017-03-25 03:00:00,1010.0,0.0,8.0,9.0,2.0,0.707107,0.707107,1.0,6.123234e-17,0.937752,...,278.0,271.0,263.0,256.0,288.0,320.0,352.0,245.0,138.0,31.0
2017-03-25 04:00:00,1011.0,0.0,10.0,10.0,4.0,0.866025,0.500000,1.0,6.123234e-17,0.937752,...,280.0,278.0,271.0,263.0,256.0,288.0,320.0,352.0,245.0,138.0


In [87]:
errors = sliding_window(data)

Fitting window 1...


C:\Users\danie\AppData\Local\Temp\ipykernel_11036\844988700.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_features[f'load_lag_{lag}'] = forecast_features[f'load_lag_{lag-1}']
C:\Users\danie\AppData\Local\Temp\ipykernel_11036\844988700.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_features['load_lag_1'] = forecast


Fitting window 2...


C:\Users\danie\AppData\Local\Temp\ipykernel_11036\844988700.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_features[f'load_lag_{lag}'] = forecast_features[f'load_lag_{lag-1}']
C:\Users\danie\AppData\Local\Temp\ipykernel_11036\844988700.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_features['load_lag_1'] = forecast


Fitting window 3...


...

Fitting window 8...


C:\Users\danie\AppData\Local\Temp\ipykernel_11036\844988700.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_features[f'load_lag_{lag}'] = forecast_features[f'load_lag_{lag-1}']
C:\Users\danie\AppData\Local\Temp\ipykernel_11036\844988700.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_features['load_lag_1'] = forecast


Fitting window 9...


C:\Users\danie\AppData\Local\Temp\ipykernel_11036\844988700.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_features[f'load_lag_{lag}'] = forecast_features[f'load_lag_{lag-1}']
C:\Users\danie\AppData\Local\Temp\ipykernel_11036\844988700.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_features['load_lag_1'] = forecast


KeyboardInterrupt: 

Multi-Step Forecasting:

To make multi-step forecasts, you need to implement a forecasting loop that iteratively predicts each future time point within the forecast horizon. The general steps within this loop include:
- Provide the model with input features for the current time step.
- Use the model to predict the value for the current time step.
- Update the input features for the next time step by incorporating the actual value just predicted (autoregressive approach).
- Repeat these steps for each time step within the forecast horizon

model.predict(x) \
x should include: 
- lagged loads
- lagged weather data
- lead weather data